In [ ]:
!pip install ciw 
!pip install chart_studio
!pip3 install plotly

In [45]:
import ciw
from ciw.dists import *
import math
import json
from collections import defaultdict
import chart_studio.plotly as py
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import display, Markdown, HTML
from sklearn.neighbors import KernelDensity

import datetime
import zipfile

import pickle
import pandas as pd
import numpy as np
import random

import plotly
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
######################################################
# Constants

TICKS_IN_HOUR = 60
TICKS_IN_DAY = 24 * TICKS_IN_HOUR
TICKS_IN_YEAR = 365 * TICKS_IN_DAY
CUR_COLOR = '#2E7D32'
PREV_COLOR = '#A5D6A7'


In [46]:
CLASS_COLORS = {0: '#B22222', 1: '#4169E1', 2: '#FF69B4'}
CLASS_NAMES = {0: 'HIGH', 1: 'MEDIUM', 2: 'LOW'}

def build_ts(values, start=pd.Timestamp.now(tz="America/New_York"), 
             resample='5T', line=dict(color=CUR_COLOR,width=1), name=None, showlegend=False):
    tmp = pd.DatetimeIndex([ start + pd.Timedelta(datetime.timedelta(minutes=rec)) for rec in values ])
    arrival = pd.Series(np.ones(tmp.size),index=tmp)
    f = arrival.resample(resample).sum()
    return go.Scatter(x=f.index, y=f,
                      showlegend=showlegend,
                      line=line,
                      name=name)


def _build_lat_df(q):
    df = pd.DataFrame([{
        'date': datetime.datetime.now() + datetime.timedelta(minutes=rec.arrival_date),
        'latency': rec.exit_date -  rec.arrival_date,
        'customer_class': rec.customer_class
    } for rec in  q.get_all_records()])
    df['date'] = pd.to_datetime(df['date']).dt.round('s')
    df.sort_values(by='date', inplace=True)
    return df

def build_dashboard(q, prev_q=None, 
                    wait_time_slo=15, 
                    latency_slo=30,
                    title="Run Data",
                    use_customer_classes=False,
                    plot_prev_arrival=True,
                    plot_prev_latency=True,
                    show_slo=False,
                    util_index=1):

    #
    # Utilization [gauge]
    # Latency [bargraph]
    # 
    #
    # If we didn't get a prev_q var we'll plot latency 
    num_rows = 5
    sub_titles = ["Throughput","Utilization", "Latency Distribution", "Wait Time Distribution","Arrival Rate", "Latency"]
    specs = [ [{"type": "domain"},{"type": "domain"}],
              [  {"type": "xy","colspan":2}, None  ],                                   
              [  {"type": "xy","colspan":2}, None  ],
              [  {"type": "xy","colspan":2}, None  ],
              [  {"type": "xy","colspan":2}, None  ] ]
                                   
    num_cols = 2
    util_inv = -1
    ut_indicator_inv = None
    if len(q.nodes) == 5:
        num_cols = 3
        sub_titles = ["Throughput","Utilization (Triage)", "Utilization (Investigation)", "Latency Distribution", "Wait Time Distribution","Arrival Rate", "Latency"]
        specs = [ [{"type": "domain"},{"type": "domain"},{"type": "domain"}],
              [  {"type": "xy","colspan":3}, None, None  ],                                   
              [  {"type": "xy","colspan":3}, None, None  ],
              [  {"type": "xy","colspan":3}, None, None  ],
              [  {"type": "xy","colspan":3}, None, None  ] ]
        util_inv = q.nodes[2].server_utilisation * 100
        
    
    main_fig = make_subplots(rows=num_rows, cols=num_cols,
                             row_titles=[None,"Latency"],
                             subplot_titles=sub_titles,
                             specs=specs)

    throughput = len(q.get_all_records()) / q.current_time
    
    utilization = q.nodes[util_index].server_utilisation * 100

    if prev_q != None:
        prev_throughput = len(prev_q.get_all_records()) / prev_q.current_time
        prev_utilization = prev_q.nodes[util_index].server_utilisation * 100 # [ node.server_utilisation for node in prev_q.nodes[1:-1] ][0]

        tp_indicator = go.Indicator(mode = "number+delta",
                                    value = throughput,
                                    number = {'suffix': "/min"},
                                    delta = {'position':'bottom','reference': prev_throughput},
                                    domain = {'x': [0, 1], 'y': [0, 1]})
        ut_indicator = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = utilization,
                                    mode = "gauge+number+delta",
                                    delta = {'reference' : prev_utilization},
                                    number = {'suffix' : '%'},
                                    gauge = {'axis': {'range': [0, 100]},
                                             'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                         {'range': [70, 90], 'color': "gray"}],
                                             'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                             'thickness': 0.75, 'value': 90}})
        
        if num_cols == 3:
            prev_util_inv = 0
            if len(prev_q.nodes) == 5:
                prev_util_inv = prev_q.nodes[2].server_utilisation * 100
                # Previous triage utilization
                prev_utilization = q.nodes[1].server_utilisation * 100
                
            ut_indicator = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                        value = utilization,
                                        mode = "gauge+number+delta",
                                        delta = {'reference' : prev_utilization},
                                        number = {'suffix' : '%'},
                                        gauge = {'axis': {'range': [0, 100]},
                                                 'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                             {'range': [70, 90], 'color': "gray"}],
                                                 'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                                 'thickness': 0.75, 'value': 90}})
            
            ut_indicator_inv = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                        value = util_inv,
                                        mode = "gauge+number+delta",
                                        delta = {'reference' : prev_util_inv},
                                        number = {'suffix' : '%'},
                                        gauge = {'axis': {'range': [0, 100]},
                                                 'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                             {'range': [70, 90], 'color': "gray"}],
                                                 'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                                 'thickness': 0.75, 'value': 90}})
    else:
        tp_indicator = go.Indicator(mode = "number",
                                    value = throughput,
                                    number = {'suffix': "/min"},
                                    domain = {'x': [0, 1], 'y': [0, 1]})
        ut_indicator = go.Indicator(domain = {'x': [0, 1], 'y': [0, 1]},
                                    value = utilization,
                                    mode = "gauge+number",
                                    number = {'suffix' : '%'},
                                    gauge = {'axis': {'range': [0, 100]},
                                              'steps' : [ {'range': [0, 70], 'color': "lightgray"},
                                                          {'range': [70, 90], 'color': "gray"}],
                                              'threshold' : { 'line': {'color': "red", 'width': 4}, 
                                                              'thickness': 0.75, 'value': 90}})

    main_fig.add_trace(tp_indicator,1,1)
    main_fig.add_trace(ut_indicator,1,2)
    if ut_indicator_inv is not None:
        main_fig.add_trace(ut_indicator_inv,1,3)
        

    #
    # Latency Distribution
    #    
    late_data = [ rec.exit_date - rec.arrival_date for rec in q.get_all_records() if rec.exit_date > rec.arrival_date ]
    counts, bins = np.histogram(late_data, bins=20)
    
    top_y = max(counts)
    #print("TOP Y LAT CURR Q: ", top_y)
    #print("\tCOUNTS: ", counts)
    #print("\tBINS: ", bins)
    
    h = go.Histogram(
                     x=late_data,
                     #y=counts,
                     name="Current", 
                     marker_color=CUR_COLOR,
                     bingroup=1,
                     nbinsx=20
                    )


    main_fig.add_trace(
        h,
                    row=2,col=1)

    if prev_q != None:
        prev_late_data = [ rec.exit_date - rec.arrival_date for rec in prev_q.get_all_records() if rec.exit_date > rec.arrival_date ]
        #print(len(prev_late_data))
        counts, bins = np.histogram(prev_late_data, bins=20)
        #print("\tCOUNTS: ", counts)
        #print("\tBINS: ", bins)
        if max(counts) > top_y:
            top_y = max(counts)
            
        main_fig.add_trace(
            go.Histogram(
                x=prev_late_data, 
                #y=counts,
                name="Prev", 
                marker_color=PREV_COLOR,
                bingroup=1,
                nbinsx=20
            ),
            row=2,col=1)
        
        

    
    #print("TOP Y PREV Q: ", top_y)
    #print("\n")
   

    if show_slo is True:
        # SLO Line / Text
        main_fig.add_trace(go.Scatter(x=[wait_time_slo,wait_time_slo], y=[0,top_y],
                                        mode="lines+text",
                                        name="SLO",
                                        text=[None, "SLO"],
                                        textposition="bottom center",
                                        line=dict(color='red',width=3)), 
                                        row=2, col=1)

    

    #
    # Wait time distribution
    #
    wait_data = [ rec.waiting_time for rec in q.get_all_records() if rec.waiting_time > 0]
    counts, bins = np.histogram(wait_data, bins=20)
    top_y = max(counts)
    
    main_fig.add_trace(go.Histogram(x=wait_data, marker_color=CUR_COLOR,
                                    showlegend=False,bingroup=2),
                       row=3,col=1)

    if prev_q != None:
        prev_wait_data = [ rec.waiting_time for rec in prev_q.get_all_records() if rec.waiting_time > 0 ]

        main_fig.add_trace(go.Histogram(x=prev_wait_data, showlegend=False, marker_color=PREV_COLOR,
                                        bingroup=2),
                          row=3,col=1)
        
        counts, bins = np.histogram(prev_wait_data, bins=20)
        if max(counts) > top_y:
            top_y = max(counts)

    
    if show_slo is True:
        # SLO Line + Text
        main_fig.add_trace(go.Scatter(x=[wait_time_slo,wait_time_slo], 
                                      y=[0,top_y],
                                      mode="lines+text",
                                      name="SLO",
                                      text=[None, "SLO"],
                                      showlegend=False,
                                      textposition="bottom center",
                                      line=dict(color='red',width=3)), 
                                      row=3, col=1)

    
    main_fig.update_layout(barmode="overlay", bargap=0.1, 
                           height=900, width=800,
                           title={ 'text': title,
                           'y':0.98, 'x':0.5, 'xanchor': 'center',
                           'font': {'size': 20}, 'yanchor': 'top'})

    #
    # Arrival Timeseries
    #
    if use_customer_classes is False:
        ts = build_ts([ rec.arrival_date for rec in q.get_all_records() ],
                      line=dict(color=CUR_COLOR,width=1))
        lat_times = [ rec.service_time for rec in q.get_all_records() ]
        main_fig.add_trace(ts, row=4, col=1)
        
        
        #main_fig.add_trace(go.Scatter(x=np.arange(len(lat_times)), y=lat_times,
        #                              showlegend=False,
        #                              line=dict(color=CUR_COLOR,width=1)), 
        #                   row=5, col=1)
        
        df = _build_lat_df(q)
        main_fig.add_trace(go.Scatter(x=df['date'], y=df['latency'],
                                              showlegend=False,
                                              line=dict(color=CUR_COLOR,width=1)),
                                   row=5, col=1)
    else:
        # Hard coding this... YOLO ..
        
        df = _build_lat_df(q)
        
        for customer_class in [0, 1, 2]:
            ts = build_ts([ rec.arrival_date for rec in q.get_all_records() if rec.customer_class == customer_class],
                      line=dict(color=CLASS_COLORS[customer_class],width=1), name=CLASS_NAMES[customer_class], showlegend=True)
            
            #lat_times = [ rec.service_time for rec in q.get_all_records() if rec.customer_class == customer_class]
            main_fig.add_trace(ts, row=4, col=1)
            
            
            #main_fig.add_trace(go.Scatter(x=np.arange(len(lat_times)), y=lat_times, showlegend=False,
            #                              line=dict(color=CLASS_COLORS[customer_class],width=1)), 
            #                   row=5, col=1)
            tmp = df[(df['customer_class'] == customer_class)]
            main_fig.add_trace(go.Scatter(x=tmp['date'], y=tmp['latency'],
                                              showlegend=False,
                                              line=dict(color=CLASS_COLORS[customer_class],width=1)),
                                   row=5, col=1)
            
        
                 
    if prev_q is not None:
        if use_customer_classes is False:
            if plot_prev_arrival:
                ts2 = build_ts([ rec.arrival_date for rec in prev_q.get_all_records() ],
                               line=dict(color=PREV_COLOR,width=1))
                main_fig.add_trace(ts2, row=4, col=1)

            if plot_prev_latency:
                df = _build_lat_df(prev_q)
                main_fig.add_trace(go.Scatter(x=df['date'], y=df['latency'],
                                              showlegend=False,
                                              line=dict(color=PREV_COLOR,width=1)),
                                   row=5, col=1)
        else:
            # Hard coding this... YOLO ..
            df = _build_lat_df(prev_q)
            for customer_class in [0, 1, 2]:
                if plot_prev_arrival:
                    ts2 = build_ts([ rec.arrival_date for rec in prev_q.get_all_records() if rec.customer_class == customer_class],
                                   line=dict(color=CLASS_COLORS[customer_class],width=1))
                    main_fig.add_trace(ts2, row=4, col=1)

                if plot_prev_latency:
                    
                    #lat_times2 = [ rec.service_time for rec in prev_q.get_all_records() if rec.customer_class == customer_class]
                    #main_fig.add_trace(go.Scatter(x=np.arange(len(lat_times2)), y=lat_times, showlegend=False,
                    #                              line=dict(color=CLASS_COLORS[customer_class],width=1)),
                    #                   row=5, col=1)
                    tmp = df[(df['customer_class'] == customer_class)]
                    main_fig.add_trace(go.Scatter(x=tmp['date'], y=tmp['latency'],
                                              showlegend=False,
                                              line=dict(color=CLASS_COLORS[customer_class],width=1)),
                                   row=5, col=1)
    main_fig['layout']['annotations'][0].update({'y':1.03})
    main_fig['layout']['annotations'][1].update({'y':1.03})
    if ut_indicator_inv is not None:
        main_fig['layout']['annotations'][2].update({'y':1.03})
    return main_fig

# All your queues belong to us: Optimizing Human in the Loop
* Matt Peters (CPO)
* Peter Silberman (CTO)
![title foo](img/expel_logo.png "Title")

<h1>Introduction</h1>

What we're going to cover in this talk:

<ul>
    <li>General Idea of Queues / Queueing</li>
    <li>CIW Simulation Framework</li>
    <li>Application to Security Operations</li>
</ul>

<h1>Queues Explained - Parameters</h1>

![caption](img/queue-intro.png)

There are several parameters that define a queueing system
<ul>
    <li><b>Arrival Rate</b> - &lambda; - How fast does work arrive?</li>
    <li><b>Service Rate</b> - &mu; - How long does each job take to do?</li>
    <li><b>Number of servers</b> - How many jobs can we do at the same time</li>
    <li><b>Classes of service</b> - what order to we process work in?</li>
</ul>

![caption](img/queue-intro-measurement.png)

There are several measurement points you want to have
<ul>
    <li><b>[2] - [1]</b> - How long is a job waiting to be serviced?
    <li><b>[3] - [4]</b> - How long is a job taking to be serviced?
</ul>

<h1>Queues Explained - Observation</h1>

![caption](img/arrival-times.png)    
    
If we're in the middle, then we can try to optimize for:

<ul>
    <li><b>Throughput</b> - How many jobs per unit time can we do?</li>
    <li><b>Latency</b> - What's the end-to-end time for a job to travel through the system?</li>
    <li><b>Utilization</b> - How busy is each server in our system?</li>
</ul>



<h1>Queues In The Real World</h1>

![caption](img/queue-stock.jpg)

<h1>Simulating Queues - A Simple Example</h1>

<img src="img/queue-demo-1-new.png" alt="drawing"/>


In [47]:
network = ciw.create_network(arrival_distributions=[ Deterministic(4) ], 
                             service_distributions=[ Deterministic(5) ], 
                             number_of_servers=[1]) 

Q1 = ciw.Simulation(network)
Q1.simulate_until_max_time(1440)

In [48]:
dashboard = build_dashboard(Q1, title='Arrival Rate (4) = Service Rate (5)')
dashboard.show()

<h1>What Happens if arrival or service aren't constant?</h1>

<img src="img/queue-random-functions-new.png" alt="drawing"/>



In [49]:
# Note: We changed our distributions below -- they are now exponentially distributed around
#       means of 4 and 5 min, respectively.
network = ciw.create_network(arrival_distributions=[ ciw.dists.Exponential(1.0/5) ],
                             service_distributions=[ ciw.dists.Exponential(1.0/4) ],
                             number_of_servers=[1])
Q2 = ciw.Simulation(network)
Q2.simulate_until_max_time(1440)

In [50]:
# Display The simulation Results Via Dashboard:
dashboard = build_dashboard(Q2, title='Arrval (Exp(5)) ~= Service Rate (Exp(4))', prev_q=Q1)
dashboard.show()

<h1>So What? How does this apply to me?</h1>

<img src="img/soc-as-queues.png" alt="drawing" width="500" height="500"/>

<b>Your SOC/SRE/Dev team is a queueing system. Understanding the parameters allows you to tune the output</b>

<ul>
    <li><b>Utilization</b> - This is your team, which usually feels like they're at 110%</li>
    <li><b>Throughput</b> - How many things can you triage per unit time</li>
    <li><b>Latency</b> - This is your SLO/SLA - how long until you see something?</li>
</ul>



<h1>Our Example SOC</h1>

<img src="img/our-soc-new.png" alt="drawing" width="700"/>

    with zipfile.ZipFile("arr_times_csv.zip").open('arr_times.csv') as fd:
        for line in fd.readlines()[:3]:
            print(line)

<h1>Building A Historical Distribution</h1>

<img src="img/historical-distribution.png" alt="drawing" width="700"/>

# A Historical Distribution
    class HistoricalDistribution(ciw.dists.Distribution):
        def __init__(self, sev='TOTAL',
                           filename='arr_times.pickle', 
                           adjustment_factor=0.0):
                           
            self.dists = defaultdict(dict)

            counts = get_arr_counts(filename, sev=sev)
            for day in counts.keys():
                for hr, avg_per_min in counts[day].items():
                    self.dists[day][hr] = ciw.dists.Exponential(avg_per_min - \
                                                (avg_per_min * adjustment_factor))

        def sample(self, t, ind=None):
            day    = math.floor((t / TICKS_IN_DAY) % 7)
            hour   = math.floor((t / TICKS_IN_HOUR) % 24)
            sample = self.dists[day][hour].sample(t,ind)
            return sample



<h1>Modeling A Service Distribution</h1>

<img src="img/workflow.png" alt="drawing" width="700"/>

<img src="img/service-dist-code.png" alt="drawing" width="700"/>

# Putting That All Together


    TIME_TO_TRIAGE      = 4
    TIME_TO_INVESTIGATE = 20
    TIME_TO_REPORT      = 10
    NUM_SOC_WORKERS     = 1
    SIMULATION_TIME     = 24*60

    class HistoricalDistribution(ciw.dists.Distribution):

        def __init__(self, sev='TOTAL', 
                     filename='arr_times.pickle', 
                     adjustment_factor=0.0):
                     
            self.dists = defaultdict(dict)

            counts = get_arr_counts(filename, sev=sev)
            for day in counts.keys():
                for hr, avg_per_min in counts[day].items():
                    rate = avg_per_min - (avg_per_min * adjustment_factor)
                    self.dists[day][hr] = Exponential(rate)

        def sample(self, t, ind=None):
            day    = math.floor((t / TICKS_IN_DAY) % 7)
            hour   = math.floor((t / TICKS_IN_HOUR) % 24)
            sample = self.dists[day][hour].sample(t,ind)
            return sample

    class ServiceDistribution(ciw.dists.Distribution):
        def __init__(self, avg_time_to_triage, 
                     avg_time_to_investigate,
                     avg_time_to_report, 
                     prob_of_inv=.2):
                     
            self.time_to_triage      = Exponential(1/avg_time_to_triage)
            self.time_to_investigate = Exponential(1/avg_time_to_investigate)
            self.time_to_report      = Exponential(1/avg_time_to_report)
            self.prob_of_inv         = prob_of_inv

        def sample(self, t, ind=None):
            total_time = self.time_to_triage.sample()
            if random.random() < self.prob_of_inv:
                total_time += self.time_to_investigate.sample()
                total_time += self.time_to_report.sample()

            return total_time            
            
    svc_d = ServiceDistribution(TIME_TO_TRIAGE,
                                TIME_TO_INVESTIGATE,
                                TIME_TO_REPORT)

    # Create and Run the Simulation
    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ svc_d ],
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q_basic_soc = ciw.Simulation(network)
    Q_basic_soc.simulate_until_max_time(SIMULATION_TIME)
    
    dashboard = build_dashboard(Q_basic_soc, title='Basic SOC')
    dashboard.show()

In [51]:
TIME_TO_TRIAGE      = 4
TIME_TO_INVESTIGATE = 20
TIME_TO_REPORT      = 10
NUM_SOC_WORKERS     = 1
SIMULATION_TIME     = 24*60

def get_arr_counts(filename, sev='TOTAL'):
    with open(filename, 'rb') as fd:
        dd = pickle.load(fd)

    counts = {day: {hour : 0.0 for hour in range(24)} for day in range(7)}    
    for (day, hr_sevs) in dd.items():
        for hr in hr_sevs.keys():
            cnts = dd[day][hr][sev]
            counts[day][hr] =  ((sum(cnts) / 3120)) # 52*60
    return counts


class HistoricalDistribution(ciw.dists.Distribution):

    def __init__(self, sev='TOTAL', filename='arr_times.pickle', adjustment_factor=0.0):
        self.dists = defaultdict(dict)
        
        counts = get_arr_counts(filename, sev=sev)
        for day in counts.keys():
            for hr, avg_per_min in counts[day].items():
                self.dists[day][hr] = ciw.dists.Exponential(avg_per_min -(avg_per_min * adjustment_factor) )
        
    def sample(self, t, ind=None):
        day    = math.floor((t / TICKS_IN_DAY) % 7)
        hour   = math.floor((t / TICKS_IN_HOUR) % 24)
        sample = self.dists[day][hour].sample(t,ind)
        return sample

class ServiceDistribution(ciw.dists.Distribution):
    def __init__(self, avg_time_to_triage, avg_time_to_investigate,
                 avg_time_to_report, prob_of_inv=.2):
        self.time_to_triage      = Exponential(1/avg_time_to_triage)
        self.time_to_investigate = Exponential(1/avg_time_to_investigate)
        self.time_to_report      = Exponential(1/avg_time_to_report)
        self.prob_of_inv         = prob_of_inv

    def sample(self, t, ind=None):
        total_time = self.time_to_triage.sample()
        if random.random() < self.prob_of_inv:
            total_time += self.time_to_investigate.sample()
            total_time += self.time_to_report.sample()

        return total_time            

network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ 
                                     ServiceDistribution(
                                                     TIME_TO_TRIAGE,
                                                     TIME_TO_INVESTIGATE,
                                                     TIME_TO_REPORT) 
                                     ],
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_basic_soc = ciw.Simulation(network)
Q_basic_soc.simulate_until_max_time(SIMULATION_TIME)

In [52]:
dashboard = build_dashboard(Q_basic_soc, title='Basic SOC', show_slo=True)
dashboard.show()

<h1>Our Example SOC - Adding Capacity</h1>

<img src="img/our-soc-capacity.png" alt="drawing" width="600"/>

Here we add staff, so the number of servers (SOC workers) increases.

    NUM_SOC_WORKERS = 2

    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ 
                                     ServiceDistribution(TIME_TO_TRIAGE,
                                                         TIME_TO_INVESTIGATE,
                                                         TIME_TO_REPORT) 
                                 ],
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(SIMULATION_TIME)

In [53]:
NUM_SOC_WORKERS = 2

network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ ServiceDistribution(TIME_TO_TRIAGE,
                                                                         TIME_TO_INVESTIGATE,
                                                                         TIME_TO_REPORT) ],
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_add_cap = ciw.Simulation(network)

Q_add_cap.simulate_until_max_time(SIMULATION_TIME)

In [54]:
dashboard = build_dashboard(Q_add_cap, prev_q=Q_basic_soc, title='Added Capacity vs Basic SOC', show_slo=True)
dashboard.show()

<h1>Our Example SOC - Training</h1>

<img src="img/our-soc-training.png" alt="drawing" width="700"/>

Here we add training, so the probability of investigation is reduced by 10%, as well as the time it takes by 20%.

    TIME_TO_INVESTIGATE  *= .20
    TIME_TO_REPORT       *= .20
    PROB_OF_INVESTIGATION = .18

    network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                                 service_distributions=[ 
                                         ServiceDistribution(TIME_TO_TRIAGE,
                                                             TIME_TO_INVESTIGATE,
                                                             TIME_TO_REPORT,
                                                             PROB_OF_INVESTIGATION) 
                                                  ],
                                number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)

    Q.simulate_until_max_time(simulation_time)

In [55]:
TIME_TO_INVESTIGATE  *= .20
TIME_TO_REPORT       *= .20
PROB_OF_INVESTIGATION = .18

network = ciw.create_network(arrival_distributions=[ HistoricalDistribution() ],
                             service_distributions=[ 
                                             ServiceDistribution(TIME_TO_TRIAGE,
                                                                 TIME_TO_INVESTIGATE,
                                                                 TIME_TO_REPORT,
                                                                 PROB_OF_INVESTIGATION) ],
                            number_of_servers=[ NUM_SOC_WORKERS ])

Q_train = ciw.Simulation(network)
Q_train.simulate_until_max_time(SIMULATION_TIME)

In [56]:
dashboard = build_dashboard(Q_train, prev_q=Q_add_cap, title='Added Capacity vs Training', show_slo=True)
dashboard.show()

<h1>Our Example SOC - Tuning</h1>

<img src="img/our-soc-tuning.png" alt="drawing" width="800"/>

In [57]:
ADJUSTMENT_FACTOR = .15
PROB_OF_INVESTIGATION  = 0.2 * .1
network = ciw.create_network(arrival_distributions=[ HistoricalDistribution(adjustment_factor=ADJUSTMENT_FACTOR) ],
                             service_distributions=[ ServiceDistribution(TIME_TO_TRIAGE,
                                                                         TIME_TO_INVESTIGATE,
                                                                         TIME_TO_REPORT,
                                                                         PROB_OF_INVESTIGATION) ],

                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_tuning = ciw.Simulation(network)
Q_tuning.simulate_until_max_time(SIMULATION_TIME)

In [58]:
dashboard = build_dashboard(Q_tuning, prev_q=Q_train, show_slo=True, title='Tuning vs Training')
dashboard.show()

<h1>Our Example SOC - Service Classes</h1>

<img src="img/our-soc-classes.png" alt="drawing" width="700"/>

Here we add service classes, so our alerts are now associated with a severity (HIGH, MED, LOW) which are each handled in order.

    priority_classes={ 'Class 0' : 0,
                       'Class 1' : 1,
                       'Class 2' : 2 }

    arrival_distributions = { 'Class 0' :  [ HistoricalDistribution(sev='HIGH', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ],
                              'Class 1' :  [ HistoricalDistribution(sev='MEDIUM', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ],
                              'Class 2' :  [ HistoricalDistribution(sev='LOW', 
                                                    adjustment_factor=ADJUSTMENT_FACTOR) ] }


    service_distributions={ 'Class 0' : [ ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ],
                            'Class 1' : ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ],
                            'Class 2' : ServiceDistribution(TIME_TO_TRIAGE,
                                                                TIME_TO_INVESTIGATE,
                                                                TIME_TO_REPORT,
                                                                PROB_OF_INVESTIGATION) ] }

    network = ciw.create_network(arrival_distributions=arrival_distributions,
                                 service_distributions=service_distributions,
                                 priority_classes=priority_classes,
                                 number_of_servers=[ NUM_SOC_WORKERS ])

    Q = ciw.Simulation(network)
    Q.simulate_until_max_time(SIMULATION_TIME)

In [59]:
priority_classes={ 'Class 0' : 0,
                   'Class 1' : 1,
                   'Class 2' : 2 }

arrival_distributions = { 'Class 0' :  [ HistoricalDistribution(sev='HIGH', adjustment_factor=ADJUSTMENT_FACTOR) ],
                          'Class 1' :  [ HistoricalDistribution(sev='MEDIUM', adjustment_factor=ADJUSTMENT_FACTOR) ],
                          'Class 2' :  [ HistoricalDistribution(sev='LOW', adjustment_factor=ADJUSTMENT_FACTOR) ] }


service_distributions={ 'Class 0' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ],
                        'Class 1' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ],
                        'Class 2' : [ ServiceDistribution(TIME_TO_TRIAGE, TIME_TO_INVESTIGATE,
                                                          TIME_TO_REPORT, PROB_OF_INVESTIGATION) ] }

network = ciw.create_network(arrival_distributions=arrival_distributions,
                             service_distributions=service_distributions,
                             priority_classes=priority_classes,
                             number_of_servers=[ NUM_SOC_WORKERS ])

Q_svc_cls = ciw.Simulation(network)
Q_svc_cls.simulate_until_max_time(SIMULATION_TIME)

In [60]:
dashboard = build_dashboard(Q_svc_cls, prev_q=Q_tuning, show_slo=True, title='Service Classes vs Tuning')
dashboard.show()

df = _build_lat_df(Q_svc_cls)

fig = go.Figure(data=[
    go.Histogram(x=df[df['customer_class'] == 0]['latency'], name='high severity'),
    go.Histogram(x=df[df['customer_class'] == 2]['latency'], name='low severity')])
fig.update_layout(title='Latency of high vs low severity alerts')
fig.show()

<h1>Our Example SOC - Tiering</h1>

<img src="img/our-soc-tiering-img.png" alt="drawing" width="800"/>
<img src="img/our-soc-tiering-code.png" alt="drawing" width="800"/>

In [61]:
class InvestigationDistribution(ciw.dists.Distribution):
    def __init__(self, avg_time_to_investigate, avg_time_to_report):
        self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
        self.time_to_report = ciw.dists.Exponential(1/avg_time_to_report)

    def sample(self, t, ind=None):
        return self.time_to_investigate.sample() + self.time_to_report.sample()

number_of_servers = [ 2, 1, 1 ]

#arrival_distributions = [ HistoricalDistribution(ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ]

arrival_per_class = { 'Class 0' : [HistoricalDistribution(sev='HIGH', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ],
                      'Class 1' : [HistoricalDistribution(sev='MEDIUM', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ],
                      'Class 2' : [HistoricalDistribution(sev='LOW', adjustment_factor=ADJUSTMENT_FACTOR), NoArrivals(), NoArrivals() ] }

service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                          InvestigationDistribution(TIME_TO_INVESTIGATE,TIME_TO_REPORT), 
                          Deterministic(0) ]

service_per_class = { 'Class 0' : service_distributions,
                      'Class 1' : service_distributions,
                      'Class 2' : service_distributions }

std_routing = [ [0.0, PROB_OF_INVESTIGATION, 1-PROB_OF_INVESTIGATION ],
                [0.0, 0.0, 1.0],
                [0.0, 0.0, 0.0] ]

routing_matrix = { 'Class 0' : std_routing, 
                   'Class 1' : std_routing,
                   'Class 2' : std_routing }

network = ciw.create_network(arrival_distributions=arrival_per_class,
                             service_distributions=service_per_class,
                             routing=routing_matrix,
                             priority_classes=priority_classes,
                             number_of_servers=number_of_servers)

Q_tier = ciw.Simulation(network)
Q_tier.simulate_until_max_time(SIMULATION_TIME)

In [62]:
dashboard = build_dashboard(Q_tier, prev_q=Q_svc_cls, show_slo=True, title='Tiering vs Service Classes')
dashboard.show()

<h1>Our Example SOC - Automation</h1>

<img src="img/our-soc-automated.png" alt="drawing" width="800"/>

In [63]:
class AutoInvestigationDistribution(ciw.dists.Distribution):

    def __init__(self, avg_time_to_investigate, avg_time_to_report,
                 prob_of_automation=0):
        self.time_to_investigate = ciw.dists.Exponential(1/avg_time_to_investigate)
        self.time_to_report      = ciw.dists.Exponential(1/avg_time_to_report)
        self.prob_of_automation  = prob_of_automation

    def is_automated(self):
        return random.random() >= self.prob_of_automation

    def sample(self, t, ind=None):
        if self.is_automated():
            return 0
        return self.time_to_investigate.sample() + self.time_to_report.sample()

PROB_OF_AUTOMATION = 0.4

service_distributions = [ Exponential(1/TIME_TO_TRIAGE), 
                          AutoInvestigationDistribution(TIME_TO_INVESTIGATE,
                                                                    TIME_TO_REPORT,
                                                                    PROB_OF_AUTOMATION), 
                          Deterministic(0) ]


service_per_class = { 'Class 0' : service_distributions,
                      'Class 1' : service_distributions,
                      'Class 2' : service_distributions }

network = ciw.create_network(arrival_distributions=arrival_per_class,
                             service_distributions=service_per_class,
                             routing=routing_matrix,
                             priority_classes=priority_classes,
                             number_of_servers=number_of_servers)

Q_auto = ciw.Simulation(network)
Q_auto.simulate_until_max_time(SIMULATION_TIME)

In [64]:
dashboard = build_dashboard(Q_auto, prev_q=Q_tier, use_customer_classes=False, show_slo=True, title='Automation vs Tiering')
dashboard.show()

<h1>Conclusion</h1>
<img src="img/themoreyouknow.png" alt="drawing" width="800"/>
<ul>
    <li>Understand what you're optimizing for</li>
    <li>Change one thing at a time to observe effects</li>
    <li>Use this to guide real-world modifications of your process</li>
</ul>
